In [1]:
import numpy as np
import numba as nb
import twopoint_counter as tpc
from matplotlib import pyplot as plt
from numba import njit, prange
import math

In [2]:
### PV catalog
NPv = int(100000)
z = np.random.rand(NPv)*0.10
pos = np.random.random((NPv,3)) * 2000 -1000
v = np.random.randn(NPv)*400

dat_vpos = pos[np.random.choice(range(pos.shape[0]), size=100, replace=False)]
ran_vpos = pos[np.random.choice(range(pos.shape[0]), size=500, replace=False)]

dat_v = np.random.choice(v,100)
ran_v = np.random.choice(v,500)

dat_vn = np.ones(100)*1e-5
ran_vn = np.ones(500)*1e-5

dat_alpha = np.ones(100)
ran_alpha = np.ones(500)
dat_vsig = np.zeros(100)
ran_vsig = np.zeros(500)
dat_vamp = ran_vamp = 100000

In [3]:
### Den catalog
Ngal = int(100000)
pos = np.random.random((Ngal,3)) * 2000 -1000

dat_dpos = pos[np.random.choice(range(pos.shape[0]), size=100, replace=False)]
ran_dpos = pos[np.random.choice(range(pos.shape[0]), size=500, replace=False)]

dat_dn = np.ones(100)*1e-5
ran_dn = np.ones(500)*1e-5
dat_damp = ran_damp = 100000

In [4]:
s_edges=np.linspace(10,120,20)
bins_s = (np.array(s_edges[1:]) + np.array(s_edges[:-1])) / 2
mu_edges = np.linspace(-1,1,100)
bins_mu = (np.array(mu_edges[1:]) + np.array(mu_edges[:-1])) / 2
edges = (s_edges,mu_edges)

In [15]:
engine = tpc.CorrelationFunctionPV(s_edges,
                                   dat_vpos,dat_vn,dat_v,dat_vsig,dat_alpha,dat_vamp,
                                   ran_vpos,ran_vn,ran_v,ran_vsig,ran_alpha,ran_vamp
                                   )


In [26]:
engine.DD(mode='psi1')
engine.DD(mode='psi2')
engine.RR(mode='psi1')
engine.RR(mode='psi2')
engine.psi(mode='psi1')
engine.psi(mode='psi2')

array([nan, nan, nan,  0., nan,  0.,  0., nan,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.])

In [5]:
engine = tpc.CorrelationFunctionPVD(edges,
                                   dat_vpos,dat_vn,dat_v,dat_vsig,dat_alpha,dat_vamp,
                                   dat_dpos,dat_dn,dat_damp,
                                   ran_vpos,ran_vn,ran_v,ran_vsig,ran_alpha,ran_vamp,
                                   ran_dpos,ran_dn,ran_damp
                                   )

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [7]:
engine.DD()
engine.RD()
engine.DR()
engine.RR()
engine.psi3()

/renoir/aronica/SNIa/pair_counting/PairCounting/twopoint_counter.py:936: RuntimeWarning: divide by zero encountered in true_divide
  psi3 = ((    RR[2]*RR[3]/DD[2]*DD[3])* DD[0]/RR[1]
/renoir/aronica/SNIa/pair_counting/PairCounting/twopoint_counter.py:936: RuntimeWarning: invalid value encountered in true_divide
  psi3 = ((    RR[2]*RR[3]/DD[2]*DD[3])* DD[0]/RR[1]
/renoir/aronica/SNIa/pair_counting/PairCounting/twopoint_counter.py:937: RuntimeWarning: divide by zero encountered in true_divide
  -2 * RR[2]/DD[2] * RD[0]/RR[1]
/renoir/aronica/SNIa/pair_counting/PairCounting/twopoint_counter.py:937: RuntimeWarning: invalid value encountered in true_divide
  -2 * RR[2]/DD[2] * RD[0]/RR[1]
/renoir/aronica/SNIa/pair_counting/PairCounting/twopoint_counter.py:938: RuntimeWarning: divide by zero encountered in true_divide
  -2 * RR[3]/DD[3] * DR[0]/RR[1]
/renoir/aronica/SNIa/pair_counting/PairCounting/twopoint_counter.py:938: RuntimeWarning: invalid value encountered in true_divide
  -2 * RR[3]

array([[          nan,           nan,           nan, ...,           nan,
                  nan,           nan],
       [-543.91359554,           nan,           nan, ...,           nan,
                  nan,           nan],
       [          nan,           nan,           nan, ...,           nan,
                  nan,           nan],
       ...,
       [          nan,           nan,           nan, ...,           nan,
                  nan,  222.22200096],
       [          nan, -364.80682677,           nan, ...,           nan,
                  nan,           nan],
       [          nan,           nan,  560.53998488, ...,  168.6480794 ,
                  nan,           nan]])